<a href="https://colab.research.google.com/github/cyb00263/test01/blob/main/dqn_tictactoe_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing dependencies
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null

!pip -q install gym
!pip -q install pyglet
!pip -q install pyopengl
!pip -q install pyvirtualdisplay

# Start virtual display
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
#os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

E: Unable to locate package libcusparse8.0
E: Couldn't find any package by glob 'libcusparse8.0'
E: Couldn't find any package by regex 'libcusparse8.0'
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'


In [2]:
!pip install tensorflow==1.14
!pip install keras==2.2.4
!pip install keras-rl==0.4.2

     |████████████████████████████████| 109.2MB 87kB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 491kB 37.6MB/s 
     |████████████████████████████████| 3.2MB 42.0MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0
     |████████████████████████████████| 317kB 5.6MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 40kB 3.6MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48382 sha256=f61ba324982a1c36a637be89375f5d916eff7

In [3]:
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np
import copy
import random

class Practice(gym.core.Env):
    def __init__(self):
        self.n_action = 10*10
        self.board = [0] * self.n_action
        self.action_space = gym.spaces.Discrete(self.n_action) # actionの取りうる値
        self.observation_space = gym.spaces.Box(low=-1, high =1, shape=(self.n_action,)) 
        self.result = []

    def step(self, action):
      self.board, end_flg, reward = self.get_input(self.board, action)
      done = end_flg
      info = {}

      return self.board, reward, done, info

    def get_input(self, board, action):
      end_flg = 0
      reward = 0
      #AI#
      space = [i for i, x in enumerate(board) if x == 0 ]
      if action in space:
        flg = 1
        board[action] = flg
        end_flg, reward = self.judge(board)
        if end_flg == 0:
          space = [i for i, x in enumerate(board) if x == 0 ]
          ransu = int(random.choice(space))
          flg = -1
          board[ransu] = flg
          end_flg, reward = self.judge(board)
          if end_flg == 1:
            reward = -10
            self.result.append(reward)
      else:
        end_flg = 1
        reward = -10
        self.result.append(reward)

      return board, end_flg, reward

    def judge(self, board):
      end_flg = 0
      reward = 0
      mass = 10
      moku = 5
      score = []

      space = [i for i, x in enumerate(board) if x == 0 ]
      if len(space) == 0:
        end_flg = 1
        reward = 0
      else:
        board = np.array(board).reshape(10,10)
        for i in range(mass-moku+1):
          for j in range(mass-moku+1):
            for k in range(moku):
              score_r = board[i][j+k] + board[i+1][j+k] + board[i+2][j+k] + board[i+3][j+k] + board[i+4][j+k]
              score.append(score_r)
              score_c = board[i+k][j] + board[i+k][j+1] + board[i+k][j+2] + board[i+k][j+3] + board[i+k][j+4]
              score.append(score_c)
            score_dl = board[i][j] + board[i+1][j+1] + board[i+2][j+2] + board[i+3][j+3] + board[i+4][j+4]
            score.append(score_dl)
            score_dr = board[i][j+4] + board[i+1][j+3] + board[i+2][j+2] + board[i+3][j+1] + board[i+4][j]
            score.append(score_dr)
        if max(score) == 5:
          end_flg = 1
          reward = 100
        elif min(score) == -5:
          end_flg = 1
          reward = -10
        else:
          reward = min(score)/10

      return end_flg, reward


    def reset(self):
        self.board = [0] * self.n_action
        return self.board

    def render(self, mode):
        pass

    def close(self):
        pass

    def seed(self):
        pass

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

practice = Practice()
n_action = 10*10

# ニューラルネットワークの構造を定義
model = Sequential()
model.add(Flatten(input_shape=(1,) + practice.observation_space.shape))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(n_action))
model.add(Activation('linear'))
print(model.summary()) # モデルの定義をコンソールに出力
 
# モデルのコンパイル
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy(tau=1.)
dqn = DQNAgent(model=model, nb_actions=n_action, memory=memory, nb_steps_warmup=50, target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               25700     
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
Total params: 51,556
Trainable params: 51,556
Non-trainable params: 0
_________________________________________________________________
None





In [ ]:
import matplotlib.pyplot as plt

# fit の結果を取得しておく
history = dqn.fit(practice, nb_steps=500000, visualize=False, verbose=1)

dqn.save_weights('dqn_{}_weights.h5f'.format("TicTacToe"), overwrite=True)

# 結果を表示
plt.subplot(2,1,1)
plt.plot(history.history["nb_episode_steps"])
plt.ylabel("step")

plt.subplot(2,1,2)
plt.plot(history.history["episode_reward"])
plt.xlabel("episode")
plt.ylabel("reward")

plt.show()  # windowが表示されます。

Training for 500000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 93s 9ms/step - reward: -1.8880
1760 episodes - episode_reward: -10.727 [-14.800, -10.100] - loss: 362.729 - mean_absolute_error: 44.271 - mean_q: 68.141

Interval 2 (10000 steps performed)
 8054/10000 [=======================>......] - ETA: 17s - reward: -1.7371

In [ ]:
import rl.callbacks
 
# ログを記録するためのクラスの定義
class EpisodeLogger(rl.callbacks.Callback):
    def __init__(self):
        self.rewards = {}
    def on_episode_begin(self, episode, logs):
        self.rewards[episode] = []
    def on_step_end(self, step, logs):
        episode = logs['episode']
        self.rewards[episode].append(logs['reward'])
 
episode_logger = EpisodeLogger()
nb_episodes = 100
dqn.test(practice, nb_episodes=nb_episodes, visualize=False, callbacks=[episode_logger])

In [ ]:
import math
import gym
from gym import spaces, logger
from gym.utils import seeding
import numpy as np
import copy
import random

class Match(Practice):
    def __init__(self):
        self.n_action = 10*10
        self.board = [0] * self.n_action
        self.action_space = gym.spaces.Discrete(self.n_action) # actionの取りうる値
        self.observation_space = gym.spaces.Box(low=-1, high =1, shape=(self.n_action,)) 

    def step(self, action):
      self.board, end_flg, reward = self.get_input(self.board, action)
      done = end_flg
      info = {}
      return self.board, reward, done, info

    def get_input(self, board, action):
      end_flg = 0
      reward = 0
      #AI#
      space = [i for i, x in enumerate(board) if x == 0 ]
      if action in space:
        flg = 1
        board[action] = flg
        end_flg, reward = self.judge(board)
        if end_flg == 1:
          reward = 10
          print("Win")
        #human#
        else:
          print('\n')
          print(str(board[0]) + '|' + str(board[1]) + '|' + str(board[2]) + '|' + str(board[3]) + '|' + str(board[4]) + '|' + str(board[5]) + '|' + str(board[6]) + '|' + str(board[7]) + '|' + str(board[8]) + '|' + str(board[9]))
          print("-------------------")
          print(str(board[10]) + '|' + str(board[11]) + '|' + str(board[12]) + '|' + str(board[13]) + '|' + str(board[14]) + '|' + str(board[15]) + '|' + str(board[16]) + '|' + str(board[17]) + '|' + str(board[18]) + '|' + str(board[19]))
          print("-------------------")
          print(str(board[20]) + '|' + str(board[21]) + '|' + str(board[22]) + '|' + str(board[23]) + '|' + str(board[24]) + '|' + str(board[25]) + '|' + str(board[26]) + '|' + str(board[27]) + '|' + str(board[28]) + '|' + str(board[29]))
          print("-------------------")
          print(str(board[30]) + '|' + str(board[31]) + '|' + str(board[32]) + '|' + str(board[33]) + '|' + str(board[34]) + '|' + str(board[35]) + '|' + str(board[36]) + '|' + str(board[37]) + '|' + str(board[38]) + '|' + str(board[39]))
          print("-------------------")
          print(str(board[40]) + '|' + str(board[41]) + '|' + str(board[42]) + '|' + str(board[43]) + '|' + str(board[44]) + '|' + str(board[45]) + '|' + str(board[46]) + '|' + str(board[47]) + '|' + str(board[48]) + '|' + str(board[49]))
          print("-------------------")
          print(str(board[50]) + '|' + str(board[51]) + '|' + str(board[52]) + '|' + str(board[53]) + '|' + str(board[54]) + '|' + str(board[55]) + '|' + str(board[56]) + '|' + str(board[57]) + '|' + str(board[58]) + '|' + str(board[59]))
          print("-------------------")
          print(str(board[60]) + '|' + str(board[61]) + '|' + str(board[62]) + '|' + str(board[63]) + '|' + str(board[64]) + '|' + str(board[65]) + '|' + str(board[66]) + '|' + str(board[67]) + '|' + str(board[68]) + '|' + str(board[69]))
          print("-------------------")
          print(str(board[70]) + '|' + str(board[71]) + '|' + str(board[72]) + '|' + str(board[73]) + '|' + str(board[74]) + '|' + str(board[75]) + '|' + str(board[76]) + '|' + str(board[77]) + '|' + str(board[78]) + '|' + str(board[79]))
          print("-------------------")
          print(str(board[80]) + '|' + str(board[81]) + '|' + str(board[82]) + '|' + str(board[83]) + '|' + str(board[84]) + '|' + str(board[85]) + '|' + str(board[86]) + '|' + str(board[87]) + '|' + str(board[88]) + '|' + str(board[89]))
          print("-------------------")
          print(str(board[90]) + '|' + str(board[91]) + '|' + str(board[92]) + '|' + str(board[93]) + '|' + str(board[94]) + '|' + str(board[95]) + '|' + str(board[96]) + '|' + str(board[97]) + '|' + str(board[98]) + '|' + str(board[99]))
          print('\n')
          space = [i for i, x in enumerate(board) if x == 0 ]
          print(space)
          human = input()
          flg = -1
          board[int(human)] = flg
          end_flg, reward = self.judge(board)
          if end_flg == 1:
            reward = -10
            print("Win")
      else:
        end_flg = 1
        reward = -10

      return board, end_flg, reward
   
    def reset(self):
        self.board = [0] * self.n_action
        return self.board

    def render(self, mode):
        pass

    def close(self):
        pass

    def seed(self):
        pass

In [ ]:
match = Match()
import rl.callbacks
 
nb_episodes = 1
dqn.test(match, nb_episodes=nb_episodes, visualize=False)